#Create plots for comparison between pseudobulk and findallmarkers methods

In [0]:
.libPaths(c("/dbfs/home/jtrincado@almirall.com/my_r_packages/Seurat", .libPaths()))
library(dplyr)
library(openxlsx)
.libPaths(c("/dbfs/home/boriol@almirall.com/my_r_packages/bulkRNASeq_PBMCs_R4.3", .libPaths()))
library(VennDiagram)
library(EnhancedVolcano)

In [0]:
volcano_generator <- function(resultsDE, given_title) {
  library(dplyr)

  resultsDE <- as.data.frame(resultsDE)
  
  # Create annotations for volcano plot
  resultsDE0 <- resultsDE
  resultsDE0$gene_id <- rownames(resultsDE0)

  # Ensure unique row names and remove rows with missing gene id
  resultsDE0 <- resultsDE0 %>%
    distinct(gene_id, .keep_all = TRUE)
  rownames(resultsDE0) <- resultsDE0$gene_id
  
  # Determine column names for p-value and log2 fold change
  p_val_col <- if ("p_val_adj" %in% colnames(resultsDE0)) "p_val_adj" else if ("padj" %in% colnames(resultsDE0)) "padj" else "adj.P.Val"
  log2fc_col <- if ("avg_log2FC" %in% colnames(resultsDE0)) "avg_log2FC" else if ("log2FoldChange" %in% colnames(resultsDE0)) "log2FoldChange" else "logFC"
  
  top10_genes <- resultsDE0 %>%
    filter(!!sym(log2fc_col) > 1 & !!sym(p_val_col) < 0.05) %>%
    arrange(!!sym(p_val_col)) %>% top_n(10, -!!sym(p_val_col))
  
  bottom10_genes <- resultsDE0 %>%
    filter(!!sym(log2fc_col) < -1 & !!sym(p_val_col) < 0.05) %>%
    arrange(!!sym(p_val_col)) %>% top_n(10, -!!sym(p_val_col))
  
  # Plot Volcano
  volcano <- EnhancedVolcano(resultsDE0,
    lab = rownames(resultsDE0),
    x = log2fc_col,
    y = p_val_col,
    pCutoff = 0.05,
    selectLab = c(top10_genes$gene_id, bottom10_genes$gene_id),
    labSize = 5,
    drawConnectors = TRUE,
    widthConnectors = 0.5,
    colConnectors = 'black',
    title = given_title)
  volcano
}

#KC

## Reynolds

In [0]:
kc.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/reynolds_kc_LvsHC_allmarkers.xlsx")
rownames(kc.de) <- kc.de$gene
kc.de.orig <- kc.de

In [0]:
bulk.kc.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/pseudobulk/reynolds_kc_LvsHC_bulk_v2.xlsx")
rownames(bulk.kc.de) <- bulk.kc.de$gene
bulk.kc.de.orig <- bulk.kc.de

In [0]:
bulk.kc.limma <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/pseudobulk/reynolds_limma_results_kc.xlsx")
rownames(bulk.kc.limma) <- bulk.kc.limma$gene


In [0]:
kc.de.mast <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/MAST_method/reynolds_kc_LvsHC_allmarkers.xlsx")
rownames(kc.de.mast) <- kc.de.mast$gene

In [0]:
# compare the DE P-values between the single-cell level and the pseudobulk level results
names(bulk.kc.de) <- paste0(names(bulk.kc.de), ".bulk")
bulk.kc.de$gene <- rownames(bulk.kc.de)

names(kc.de) <- paste0(names(kc.de), ".sc")
kc.de$gene <- rownames(kc.de)

merge_dat <- merge(kc.de, bulk.kc.de, by = "gene")
merge_dat <- merge_dat[order(merge_dat$p_val_adj.bulk), ]

# Number of genes that are marginally significant in both; marginally significant only in bulk; and marginally significant only in single-cell
common <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.bulk > 0.5 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
only_sc <- merge_dat$gene[which(merge_dat$p_val_adj.bulk > 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
only_bulk <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                  merge_dat$p_val_adj.sc > 0.05 & 
                                  merge_dat$avg_log2FC.bulk > 0.5)]
print(paste0('# Common: ', length(common)))
print(paste0('# Only in bulk: ', length(only_bulk)))
print(paste0('# Only in single-cell: ', length(only_sc)))

In [0]:
common <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.bulk > 0.5 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
common <- common[order(merge_dat$p_val_adj.bulk[match(common, merge_dat$gene)])]
print(common[1:20])

In [0]:
kc.de.sig <- kc.de[kc.de$p_val_adj.sc < 0.05 & abs(kc.de$avg_log2FC.sc) > 1,]
bulk.kc.de.sig <- bulk.kc.de[bulk.kc.de$p_val_adj.bulk < 0.05 & abs(bulk.kc.de$avg_log2FC.bulk) > 1,]

In [0]:
# Create a list of your gene sets
gene_sets <- list(
  "SC" = rownames(kc.de.sig),
  "Bulk" = rownames(bulk.kc.de.sig)
)
options(repr.plot.width=1000, repr.plot.height=1000)

# Create a new plotting page
grid.newpage()

# Plot the Venn diagram with colors and title
venn.plot <- venn.diagram(
  x = gene_sets,
  category.names = c("FindAllMarkers", "Pseudo-Bulk"), # Use abbreviated names for legend
  filename = NULL,  # Set to NULL to plot in RStudio
  output = TRUE,
  fill = c("red", "blue"), # Add colors
  main = "Common DEGs - Reynolds - Keratinocytes", # Add title
  main.fontface = "bold", # Make title bold
  cat.dist = c(0.03, 0.03), # Adjust the distance of the category names from the circles
  main.cex = 1.3, # Increase title size
  cat.cex = 1.3, # Increase label size
  cat.pos = c(-17, 17), # Position labels more on the top
  cex = 1.3, # Increase numbers size
  cat.col = c("red", "blue") # Set label colors to match circles
)

# Display the plot
grid.draw(venn.plot)

In [0]:
kc.de.orig <- kc.de.orig %>%
    mutate(avg_log2FC = ifelse(cluster == "healthy", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign
kc.de.mast <- kc.de.mast %>%
    mutate(avg_log2FC = ifelse(cluster == "healthy", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign

In [0]:
options(repr.plot.width=1600, repr.plot.height=1000)


In [0]:
library(patchwork)

p1 <- volcano_generator(kc.de.orig, "FindAllMarkers")
p2 <- volcano_generator(kc.de.mast, "MAST")
p3 <- volcano_generator(bulk.kc.de.orig, "Pseudo-bulk DESeq2")
p4 <- volcano_generator(bulk.kc.limma, "Pseudo-bulk Limma")

combined_plot <- (p1 + p2) + plot_annotation(title = "Reynolds - Keratinocytes", 
                                             theme = theme(plot.title = element_text(hjust = 1, vjust = 2, size = 17, face = "bold")))
print(combined_plot)

In [0]:
library(patchwork)

p3 <- volcano_generator(bulk.kc.de.orig, "Pseudo-bulk DESeq2")
p4 <- volcano_generator(bulk.kc.limma, "Pseudo-bulk Limma")

combined_plot <- (p3 + p4) + plot_annotation(                                             theme = theme(plot.title = element_text(hjust = 0.5, vjust = 2, size = 17, face = "bold")))
print(combined_plot)

##KC Alkon

In [0]:
kc.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/alkon_kc_LvsHC_allmarkers.xlsx")
rownames(kc.de) <- kc.de$gene
kc.de.orig <- kc.de

In [0]:
bulk.kc.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/pseudobulk/alkon_kc_LvsHC_bulk_v2.xlsx")
rownames(bulk.kc.de) <- bulk.kc.de$gene
bulk.kc.de.orig <- bulk.kc.de

In [0]:
bulk.kc.limma <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/pseudobulk/alkon_limma_results_kc.xlsx")
rownames(bulk.kc.limma) <- bulk.kc.limma$gene

kc.de.mast <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/MAST_method/alkon_kc_LvsHC_allmarkers.xlsx")
rownames(kc.de.mast) <- kc.de.mast$gene

In [0]:
# compare the DE P-values between the single-cell level and the pseudobulk level results
names(bulk.kc.de) <- paste0(names(bulk.kc.de), ".bulk")
bulk.kc.de$gene <- rownames(bulk.kc.de)

names(kc.de) <- paste0(names(kc.de), ".sc")
kc.de$gene <- rownames(kc.de)

merge_dat <- merge(kc.de, bulk.kc.de, by = "gene")
merge_dat <- merge_dat[order(merge_dat$p_val_adj.bulk), ]

# Number of genes that are marginally significant in both; marginally significant only in bulk; and marginally significant only in single-cell
common <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.bulk > 0.5 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
only_sc <- merge_dat$gene[which(merge_dat$p_val_adj.bulk > 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
only_bulk <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                  merge_dat$p_val_adj.sc > 0.05 & 
                                  merge_dat$avg_log2FC.bulk > 0.5)]
print(paste0('# Common: ', length(common)))
print(paste0('# Only in bulk: ', length(only_bulk)))
print(paste0('# Only in single-cell: ', length(only_sc)))

In [0]:
common <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.bulk > 0.5 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
common <- common[order(merge_dat$p_val_adj.bulk[match(common, merge_dat$gene)])]
print(common[1:20])

In [0]:
kc.de.sig <- kc.de[kc.de$p_val_adj.sc < 0.05 & abs(kc.de$avg_log2FC.sc) > 1,]
bulk.kc.de.sig <- bulk.kc.de[bulk.kc.de$p_val_adj.bulk < 0.05 & abs(bulk.kc.de$avg_log2FC.bulk) > 1,]

In [0]:
options(repr.plot.width=1000, repr.plot.height=1000)


In [0]:
# Create a list of your gene sets
gene_sets <- list(
  "SC" = rownames(kc.de.sig),
  "Bulk" = rownames(bulk.kc.de.sig)
)

# Create a new plotting page
grid.newpage()

# Plot the Venn diagram with colors and title
venn.plot <- venn.diagram(
  x = gene_sets,
  category.names = c("FindAllMarkers", "Pseudo-Bulk"), # Use abbreviated names for legend
  filename = NULL,  # Set to NULL to plot in RStudio
  output = TRUE,
  fill = c("red", "blue"), # Add colors
  main = "Common DEGs - Alkon - Keratinocytes", # Add title
  cat.dist = c(0.03, 0.03), # Adjust the distance of the category names from the circles
  main.cex = 1.3, # Increase title size
  main.fontface = "bold", # Make title bold
  cat.cex = 1.3, # Increase label size
  cat.pos = c(-17, 17), # Position labels more on the top
  cex = 1.3, # Increase numbers size
  cat.col = c("red", "blue") # Set label colors to match circles
)

# Display the plot
grid.draw(venn.plot)

In [0]:
kc.de.orig <- kc.de.orig %>%
    mutate(avg_log2FC = ifelse(cluster == "HC", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign

kc.de.mast <- kc.de.mast %>%
    mutate(avg_log2FC = ifelse(cluster == "HC", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign

In [0]:
options(repr.plot.width=1600, repr.plot.height=1000)


In [0]:
library(patchwork)

p1 <- volcano_generator(kc.de.orig, "FindAllMarkers")
p2 <- volcano_generator(kc.de.mast, "MAST")
p3 <- volcano_generator(bulk.kc.de.orig, "Pseudo-bulk DESeq2")
p4 <- volcano_generator(bulk.kc.limma, "Pseudo-bulk Limma")

combined_plot <- (p1 + p2) + plot_annotation(
                                             theme = theme(plot.title = element_text(hjust = 1, vjust = 2, size = 17, face = "bold")))
print(combined_plot)

In [0]:

p3 <- volcano_generator(bulk.kc.de.orig, "Pseudo-bulk DESeq2")
p4 <- volcano_generator(bulk.kc.limma, "Pseudo-bulk Limma")

combined_plot <- (p3 + p4) + plot_annotation(                                             theme = theme(plot.title = element_text(hjust = 0.5, vjust = 2, size = 17, face = "bold")))
print(combined_plot)

#Tcells

## Reynolds

In [0]:
tcell.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/reynolds_LvsHC_tcell_allmarkers.xlsx")
rownames(tcell.de) <- tcell.de$gene
tcell.de.orig <- tcell.de

In [0]:
bulk.tcell.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/pseudobulk/reynolds_tcell_LvsHC_bulk_v2.xlsx")
rownames(bulk.tcell.de) <- bulk.tcell.de$gene
bulk.tcell.de.orig <- bulk.tcell.de

In [0]:
bulk.tcell.limma <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/pseudobulk/reynolds_limma_results_tcell.xlsx")
rownames(bulk.tcell.limma) <- bulk.tcell.limma$gene

tcell.de.mast <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/MAST_method/reynolds_LvsHC_tcell_allmarkers.xlsx")
rownames(tcell.de.mast) <- tcell.de.mast$gene

In [0]:
# compare the DE P-values between the single-cell level and the pseudobulk level results
names(bulk.tcell.de) <- paste0(names(bulk.tcell.de), ".bulk")
bulk.tcell.de$gene <- rownames(bulk.tcell.de)

names(tcell.de) <- paste0(names(tcell.de), ".sc")
tcell.de$gene <- rownames(tcell.de)

merge_dat <- merge(tcell.de, bulk.tcell.de, by = "gene")
merge_dat <- merge_dat[order(merge_dat$p_val_adj.bulk), ]

# Number of genes that are marginally significant in both; marginally significant only in bulk; and marginally significant only in single-cell
common <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.bulk > 0.5 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
only_sc <- merge_dat$gene[which(merge_dat$p_val_adj.bulk > 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
only_bulk <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                  merge_dat$p_val_adj.sc > 0.05 & 
                                  merge_dat$avg_log2FC.bulk > 0.5)]
print(paste0('# Common: ', length(common)))
print(paste0('# Only in bulk: ', length(only_bulk)))
print(paste0('# Only in single-cell: ', length(only_sc)))

In [0]:
common <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.bulk > 0.5 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
common <- common[order(merge_dat$p_val_adj.bulk[match(common, merge_dat$gene)])]
print(common[1:20])

In [0]:
tcell.de.sig <- tcell.de[tcell.de$p_val_adj.sc < 0.05 & abs(tcell.de$avg_log2FC.sc) > 1,]
bulk.tcell.de.sig <- bulk.tcell.de[bulk.tcell.de$p_val_adj.bulk < 0.05 & abs(bulk.tcell.de$avg_log2FC.bulk) > 1,]

In [0]:
options(repr.plot.width=1500, repr.plot.height=1500)


In [0]:
# Create a list of your gene sets
gene_sets <- list(
  "SC" = rownames(tcell.de.sig),
  "Bulk" = rownames(bulk.tcell.de.sig)
)

# Create a new plotting page
grid.newpage()

# Plot the Venn diagram with colors and title
venn.plot <- venn.diagram(
  x = gene_sets,
  category.names = c("FindAllMarkers", "Pseudo-Bulk"), # Use abbreviated names for legend
  filename = NULL,  # Set to NULL to plot in RStudio
  output = TRUE,
  fill = c("red", "blue"), # Add colors
  main = "Common DEGs - Reynolds - Tcells", # Add title
  cat.dist = c(0.03, 0.03), # Adjust the distance of the category names from the circles
  main.cex = 1.3, # Increase title size
  main.fontface = "bold", # Make title bold
  cat.cex = 1.3, # Increase label size
  cat.pos = c(-17, 17), # Position labels more on the top
  cex = 1.3, # Increase numbers size
  cat.col = c("red", "blue") # Set label colors to match circles
)

# Display the plot
grid.draw(venn.plot)

In [0]:
tcell.de.orig <- tcell.de.orig %>%
    mutate(avg_log2FC = ifelse(cluster == "healthy", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign
tcell.de.mast <- tcell.de.mast %>%
    mutate(avg_log2FC = ifelse(cluster == "healthy", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign

In [0]:
options(repr.plot.width=1600, repr.plot.height=1000)


In [0]:
library(patchwork)

p1 <- volcano_generator(tcell.de.orig, "FindAllMarkers")
p2 <- volcano_generator(tcell.de.mast, "MAST")
p3 <- volcano_generator(bulk.tcell.de.orig, "Pseudo-bulk DESeq2")
p4 <- volcano_generator(bulk.tcell.limma, "Pseudo-bulk Limma")

combined_plot <- (p1 + p2) + plot_annotation(
                                             theme = theme(plot.title = element_text(hjust = 1, vjust = 2, size = 17, face = "bold")))
print(combined_plot)

In [0]:
p3 <- volcano_generator(bulk.tcell.de.orig, "Pseudo-bulk DESeq2")
p4 <- volcano_generator(bulk.tcell.limma, "Pseudo-bulk Limma")

combined_plot <- (p3 + p4) + plot_annotation(
                                             theme = theme(plot.title = element_text(hjust = 1, vjust = 2, size = 17, face = "bold")))
print(combined_plot)

## Alkon

In [0]:
tcell.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/alkon_tcell_LvsHC_allmarkers.xlsx")
rownames(tcell.de) <- tcell.de$gene
tcell.de.orig <- tcell.de

In [0]:
bulk.tcell.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/pseudobulk/alkon_tcell_LvsHC_bulk_v2.xlsx")
rownames(bulk.tcell.de) <- bulk.tcell.de$gene
bulk.tcell.de.orig <- bulk.tcell.de

In [0]:
bulk.tcell.limma <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/pseudobulk/alkon_limma_results_tcell.xlsx")
rownames(bulk.tcell.limma) <- bulk.tcell.limma$gene

tcell.de.mast <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/MAST_method/alkon_LvsHC_tcell_allmarkers.xlsx")
rownames(tcell.de.mast) <- tcell.de.mast$gene

In [0]:
# compare the DE P-values between the single-cell level and the pseudobulk level results
names(bulk.tcell.de) <- paste0(names(bulk.tcell.de), ".bulk")
bulk.tcell.de$gene <- rownames(bulk.tcell.de)

names(tcell.de) <- paste0(names(tcell.de), ".sc")
tcell.de$gene <- rownames(tcell.de)

merge_dat <- merge(tcell.de, bulk.tcell.de, by = "gene")
merge_dat <- merge_dat[order(merge_dat$p_val_adj.bulk), ]

# Number of genes that are marginally significant in both; marginally significant only in bulk; and marginally significant only in single-cell
common <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.bulk > 0.5 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
only_sc <- merge_dat$gene[which(merge_dat$p_val_adj.bulk > 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
only_bulk <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                  merge_dat$p_val_adj.sc > 0.05 & 
                                  merge_dat$avg_log2FC.bulk > 0.5)]
print(paste0('# Common: ', length(common)))
print(paste0('# Only in bulk: ', length(only_bulk)))
print(paste0('# Only in single-cell: ', length(only_sc)))

In [0]:
common <- merge_dat$gene[which(merge_dat$p_val_adj.bulk < 0.05 & 
                                merge_dat$p_val_adj.sc < 0.05 & 
                                merge_dat$avg_log2FC.bulk > 0.5 & 
                                merge_dat$avg_log2FC.sc > 0.5)]
common <- common[order(merge_dat$p_val_adj.bulk[match(common, merge_dat$gene)])]
print(common[1:20])

In [0]:
tcell.de.sig <- tcell.de[tcell.de$p_val_adj.sc < 0.05 & abs(tcell.de$avg_log2FC.sc) > 1,]
bulk.tcell.de.sig <- bulk.tcell.de[bulk.tcell.de$p_val_adj.bulk < 0.05 & abs(bulk.tcell.de$avg_log2FC.bulk) > 1,]

In [0]:
options(repr.plot.width=1000, repr.plot.height=1000)


In [0]:
# Create a list of your gene sets
gene_sets <- list(
  "SC" = rownames(tcell.de.sig),
  "Bulk" = rownames(bulk.tcell.de.sig)
)

# Create a new plotting page
grid.newpage()

# Plot the Venn diagram with colors and title
venn.plot <- venn.diagram(
  x = gene_sets,
  category.names = c("FindAllMarkers", "Pseudo-Bulk"), # Use abbreviated names for legend
  filename = NULL,  # Set to NULL to plot in RStudio
  output = TRUE,
  fill = c("red", "blue"), # Add colors
  main = "Common DEGs - Alkon - T Cells", # Add title
  cat.dist = c(0.03, 0.03), # Adjust the distance of the category names from the circles
  main.cex = 1.3, # Increase title size
  main.fontface = "bold", # Make title bold
  cat.cex = 1.3, # Increase label size
  cat.pos = c(-17, 17), # Position labels more on the top
  cex = 1.3, # Increase numbers size
  cat.col = c("red", "blue") # Set label colors to match circles
)

# Display the plot
grid.draw(venn.plot)

In [0]:
tcell.de.orig <- tcell.de.orig %>%
    mutate(avg_log2FC = ifelse(cluster == "HC", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign
    tcell.de.mast <- tcell.de.mast %>%
    mutate(avg_log2FC = ifelse(cluster == "HC", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign

In [0]:
options(repr.plot.width=1600, repr.plot.height=1000)


In [0]:
library(patchwork)

p1 <- volcano_generator(tcell.de.orig, "FindAllMarkers")
p2 <- volcano_generator(tcell.de.mast, "MAST")
p3 <- volcano_generator(bulk.tcell.de.orig, "Pseudo-bulk DESeq2")
p4 <- volcano_generator(bulk.tcell.limma, "Pseudo-bulk Limma")

combined_plot <- (p1 + p2) + plot_annotation(
                                             theme = theme(plot.title = element_text(hjust = 1, vjust = 2, size = 17, face = "bold")))
print(combined_plot)

In [0]:
p3 <- volcano_generator(bulk.tcell.de.orig, "Pseudo-bulk DESeq2")
p4 <- volcano_generator(bulk.tcell.limma, "Pseudo-bulk Limma")

combined_plot <- (p3 + p4) + plot_annotation(
                                             theme = theme(plot.title = element_text(hjust = 1, vjust = 2, size = 17, face = "bold")))
print(combined_plot)